In [1]:
using DifferentialEquations
using ReactionMechanismSimulator
using PyPlot

In [2]:
outdict = readinput("chem131.rms")


[16:45:43] WARNING: not removing hydrogen atom without neighbors
[16:45:43] WARNING: not removing hydrogen atom without neighbors
┌ Warning: failed to generate StokesDiffusivity model for species vacantX
└ @ ReactionMechanismSimulator /home/ssun30/RMG-Electrocat/ReactionMechanismSimulator.jl/src/Parse.jl:352
┌ Warning: failed to generate StokesDiffusivity model for species CO2X
└ @ ReactionMechanismSimulator /home/ssun30/RMG-Electrocat/ReactionMechanismSimulator.jl/src/Parse.jl:352
┌ Warning: failed to generate StokesDiffusivity model for species CHO2X
└ @ ReactionMechanismSimulator /home/ssun30/RMG-Electrocat/ReactionMechanismSimulator.jl/src/Parse.jl:352
┌ Warning: failed to generate StokesDiffusivity model for species CO2HX
└ @ ReactionMechanismSimulator /home/ssun30/RMG-Electrocat/ReactionMechanismSimulator.jl/src/Parse.jl:352
┌ Warning: failed to generate StokesDiffusivity model for species OCX
└ @ ReactionMechanismSimulator /home/ssun30/RMG-Electrocat/ReactionMechanismSimulator.j

Dict{Any, Any} with 4 entries:
  "Solvents"              => Solvent[Solvent("water", RiedelViscosity{Float64}(…
  Set(["surface", "gas"]) => Dict{Any, Any}("Reactions"=>ElementaryReaction[vac…
  "gas"                   => Dict{Any, Any}("Species"=>Species[Species{NASA{Emp…
  "surface"               => Dict{Any, Any}("Species"=>Species[Species{NASA{Emp…

In [3]:
liqspcs = outdict["gas"]["Species"]
liqrxns = outdict["gas"]["Reactions"]
surfspcs = outdict["surface"]["Species"]
surfrxns = outdict["surface"]["Reactions"]
interfacerxns = outdict[Set(["surface", "gas"])]["Reactions"]
solv = outdict["Solvents"][1];

In [4]:
liqspcs

48-element Vector{Species}:
 Species{NASA{EmptyThermoUncertainty}, StokesDiffusivity{Float64}, EmptyHenryLawConstant, EmptyLiquidVolumetricMassTransferCoefficient}("Ar", 1, "", "[Ar]", "", NASA{EmptyThermoUncertainty}
  polys: Array{NASApolynomial}((2,))
  unc: EmptyThermoUncertainty EmptyThermoUncertainty()
, Dict("Ar" => 1), 0, StokesDiffusivity{Float64}
  r: Float64 1.9598907720674282e-10
, 1.9598907720674282e-10, 0, 0.039948, EmptyHenryLawConstant(), EmptyLiquidVolumetricMassTransferCoefficient(), "Thermo library corrected for liquid phase: primaryThermoLibrary + Solvation\ncorrection with water as solvent and solute estimated using Solute library:\nAr", false, false)
 Species{NASA{EmptyThermoUncertainty}, StokesDiffusivity{Float64}, EmptyHenryLawConstant, EmptyLiquidVolumetricMassTransferCoefficient}("He", 2, "", "[He]", "", NASA{EmptyThermoUncertainty}
  polys: Array{NASApolynomial}((2,))
  unc: EmptyThermoUncertainty EmptyThermoUncertainty()
, Dict("He" => 1), 0, StokesDiffusivi

In [5]:
fieldnames(typeof(interfacerxns[15]))

(:index, :reactants, :reactantinds, :products, :productinds, :kinetics, :electronchange, :radicalchange, :reversible, :forwardable, :pairs, :fragmentbasedreactants, :fragmentbasedproducts, :fragmentbasedreactantinds, :fragmentbasedproductinds, :comment)

In [6]:
interfacerxns

247-element Vector{ElementaryReaction}:
 vacantX+CO2<=>CO2X
 proton+CO2X<=>CHO2X
 proton+CO2X<=>CO2HX
 proton+CHO2X<=>CH2O2X
 proton+CO2HX<=>CH2O2X
 proton+OCX<=>CHOX
 proton+CHOX<=>CH2OX
 vacantX+vacantX+CO2<=>OX+OCX
 OX+proton<=>HOX
 vacantX+C=O<=>CH2OX
 ⋮
 vacantX+vacantX+O=C=CCO<=>OC[Pt]+O=C=C[Pt]
 proton+O=C=CC=[Pt]<=>CHX+C=C=O
 proton+O=C=CC#[Pt]<=>O=C=CC=[Pt]
 vacantX+vacantX+O=C=CC=O<=>OX+O=C=CC=[Pt]
 proton+OOCC#[Pt]<=>CX+COO
 vacantX+vacantX+C=COC=O<=>CH2X+O=COC=[Pt]
 proton+O=C=C=C=O.[Pt]<=>O=C=CC(=O)[Pt]
 vacantX+O=C=C=C=O<=>O=C=C=C=O.[Pt]
 proton+O=C=C=C=O.[Pt]<=>O=C=C([Pt])C=O

In [7]:
sitedensity = 2.292e-5 # Ag111
liq = IdealDiluteSolution(liqspcs,liqrxns,solv,name="liquid",diffusionlimited=true);
surf = IdealSurface(surfspcs,surfrxns,sitedensity,name="surface");

In [8]:
initialcondsliq = Dict(["proton"=>10.0^-4,
        "CO2"=>10.0^-3*10^6,
        "V"=>1.0,"T"=>300,"Phi"=>0.0,"d"=>0.0]);
AVratio = 1.0e5
initialcondssurf = Dict(["CO2X"=>0.4*sitedensity*AVratio,
        "CHO2X"=>0.1*sitedensity*AVratio,
        "CO2HX"=>0.1*sitedensity*AVratio,
        "OX"=>0.1*sitedensity*AVratio,
        "OCX"=>0.1*sitedensity*AVratio,
        "vacantX"=>0.1*sitedensity*AVratio,
        "CH2O2X"=>0.05*sitedensity*AVratio,
        "CHOX"=>0.04*sitedensity*AVratio,
        "CH2OX"=>0.01*sitedensity*AVratio,
        "A"=>1.0*AVratio,"T"=>300,"Phi"=>-1.5]);

In [9]:
domainliq,y0liq,pliq = ConstantTVDomain(phase=liq,
    initialconds=initialcondsliq,constantspecies=["proton","CO2"]);
domaincat,y0cat,pcat = ConstantTAPhiDomain(phase=surf,
    initialconds=initialcondssurf);

In [10]:
domaincat

ConstantTAPhiDomain{IdealSurface{Tuple{Arrheniusvec{Vector{Float64}, Vector{Float64}, Vector{Float64}}}, SparseArrays.SparseMatrixCSC{Float64, Int64}, Vector{Float64}, ReactionMechanismSimulator.NASAvec{EmptyThermoUncertainty}, Vector{Float64}}, Int64, Float64, Float64, Integer, Vector{Int64}}(IdealSurface{Tuple{Arrheniusvec{Vector{Float64}, Vector{Float64}, Vector{Float64}}}, SparseArrays.SparseMatrixCSC{Float64, Int64}, Vector{Float64}, ReactionMechanismSimulator.NASAvec{EmptyThermoUncertainty}, Vector{Float64}}
  name: String "surface"
  species: Array{Species}((83,))
  reactions: Array{ElementaryReaction}((1336,))
  stoichmatrix: SparseArrays.SparseMatrixCSC{Float64, Int64}
  Nrp: Array{Float64}((1336,)) [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  rxnarray: Array{Int64}((8, 1336)) [6 3 … 69 69; 5 0 … 55 64; … ; 0 0 … 0 0; 0 0 … 0 0]
  veckinetics: Tuple{Arrheniusvec{Vector{Float64}, Vector{Float64}, Vector{Float64}}}
  v

In [11]:
domainliq

ConstantTVDomain{IdealDiluteSolution{Tuple{Arrheniusvec{Vector{Float64}, Vector{Float64}, Vector{Float64}}}, SparseArrays.SparseMatrixCSC{Float64, Int64}, Vector{Float64}}, Int64, Float64, Float64, Integer, Vector{Int64}}(IdealDiluteSolution{Tuple{Arrheniusvec{Vector{Float64}, Vector{Float64}, Vector{Float64}}}, SparseArrays.SparseMatrixCSC{Float64, Int64}, Vector{Float64}}
  name: String "liquid"
  species: Array{Species}((48,))
  reactions: Array{ElementaryReaction}((8,))
  solvent: Solvent
  stoichmatrix: SparseArrays.SparseMatrixCSC{Float64, Int64}
  Nrp: Array{Float64}((8,)) [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  rxnarray: Array{Int64}((8, 8)) [29 29 … 7 7; 30 31 … 30 31; … ; 0 0 … 0 0; 0 0 … 0 0]
  veckinetics: Tuple{Arrheniusvec{Vector{Float64}, Vector{Float64}, Vector{Float64}}}
  veckineticsinds: Array{Int64}((1,)) [8]
  vecthermo: ReactionMechanismSimulator.NASAvec{EmptyThermoUncertainty}
  otherreactions: Array{ElementaryReaction}((0,))
  electronchange: Array{Float64}((

In [12]:
inter,pinter = ReactiveInternalInterfaceConstantTPhi(domainliq,
  domaincat,interfacerxns,300,AVratio*1.0);

MethodError: MethodError: no method matching ReactiveInternalInterfaceConstantTPhi(::ConstantTVDomain{IdealDiluteSolution{Tuple{Arrheniusvec{Vector{Float64}, Vector{Float64}, Vector{Float64}}}, SparseArrays.SparseMatrixCSC{Float64, Int64}, Vector{Float64}}, Int64, Float64, Float64, Integer, Vector{Int64}}, ::ConstantTAPhiDomain{IdealSurface{Tuple{Arrheniusvec{Vector{Float64}, Vector{Float64}, Vector{Float64}}}, SparseArrays.SparseMatrixCSC{Float64, Int64}, Vector{Float64}, ReactionMechanismSimulator.NASAvec{EmptyThermoUncertainty}, Vector{Float64}}, Int64, Float64, Float64, Integer, Vector{Int64}}, ::Vector{ElementaryReaction{T, Int64, V1, V2, V3, V4, Vector{Vector{String}}, Vector{Any}, Vector{Any}, Vector{Any}, Vector{Any}} where {T<:AbstractRate, V1<:AbstractArray, V2<:AbstractArray, V3<:AbstractArray, V4<:AbstractArray}}, ::Matrix{Int64}, ::SparseArrays.SparseMatrixCSC{Float64, Int64}, ::Vector{Float64}, ::Vector{Float64}, ::Vector{Float64}, ::Vector{Float64}, ::Int64, ::Float64, :

In [ ]:
@time react,y0,p = Reactor((domainliq,domaincat), (y0liq,y0cat), (0.0, 1.0e1), [inter], (pliq,pcat,pinter));


In [ ]:
@time sol = solve(react.ode,DifferentialEquations.CVODE_BDF(),abstol=1e-16,reltol=1e-6);

In [ ]:
sol.retcode

In [21]:
ssys = SystemSimulation(sol,(domainliq,domaincat,),(inter,),p);

In [ ]:
sol.t

In [ ]:
ssys.reactions[16]

In [ ]:
#Plot surface fractions
plotmolefractions(ssys.sims[2];tol=1e-9)
xlim(0.0,1)
gcf()

In [ ]:
Dict([ssys.sims[2].names[i]=>molefractions(ssys.sims[2],1.0)[i] for i in 1:length(ssys.sims[2].names)])

In [ ]:
getfluxdiagram(ssys,1;speciesratetolerance=0.4)

In [ ]:
println(ssys.names)

In [ ]:
plotrops(ssys,"CH2O2X",1;N=15,tol=0.0)

In [ ]:
plotrops(ssys,"CHO2X",1;N=10,tol=0.0)

In [ ]:
plotrops(ssys,"CO2HX",1;N=10,tol=0.0)

In [ ]:
plotrops(ssys,"OX",1;N=10,tol=0.0)
gcf()

In [ ]:
plotrops(ssys,"OCX",1.0e-6)
gcf()

In [ ]:
for (i,rxn) in enumerate(inter.reactions)
    str = getrxnstr(rxn)
    kf = inter.kfs[i]
    krev = inter.krevs[i]
    Kc = kf/krev
    println(str)
    println("kf = $kf")
    println("krev = $krev")
    println("Kc = $Kc")
end

In [ ]:
for (i,rxn) in enumerate(inter.reactions)
    str = getrxnstr(rxn)
    kf = inter.kfs[i]
    krev = inter.krevs[i]
    Kc = kf/krev
    println(str)
    println("kf = $kf")
    println("krev = $krev")
    println("Kc = $Kc")
end